In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import copy
import time

import sys
sys.path.append('../../')

from data.processing import get_data

import models
import random

from main.seir.fitting import single_fitting_cycle
from main.seir.forecast import get_forecast, forecast_all_trials, create_all_trials_csv, create_decile_csv_new
from main.seir.sensitivity import calculate_sensitivity_and_plot
import main.seir.uncertainty as uncertainty_module
from utils.generic.create_report import save_dict_and_create_report_simple
from utils.generic.config import read_config
from utils.generic.enums import Columns
from utils.fitting.loss import Loss_Calculator
from utils.generic.logging import log_wandb
from viz import plot_forecast, plot_top_k_trials, plot_ptiles
from main.seir.recovery import loss_parameter_recovery, get_top_k_with_recovery_loss

import yaml

In [ ]:
predictions_dict = {}

In [ ]:
config_filename = 'default.yaml'
config = read_config(config_filename)

In [ ]:
output_folder = '../../misc/reports/{}'.format(datetime.datetime.now().strftime("%Y_%m%d_%H%M%S"))

## Perform M1 and M2 fits

In [ ]:
best_params = []
ideal_params = []
train_losses = []
val_losses = []
n, m = 2, 1
for j in range(m):
    r = random.randint(1,100)
    for i in range(n):
        print (j, i, r)
        np.random.seed(r)
        predictions_dict['m1'] = single_fitting_cycle(**copy.deepcopy(config['fitting'])) 
        predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
        best_params.append(predictions_dict['m1']['best_params'])
        ideal_params.append(predictions_dict['m1']['ideal_params'])
        train_losses.append(predictions_dict['m1']['df_loss']['train'])
        val_losses.append(predictions_dict['m1']['df_loss']['val'])

In [ ]:
loss_parameter_recovery(best_params, ideal_params, params=['P_fatal', 'T_inc', 'T_inf', 'T_recov', 'T_recov_fatal', 'lockdown_R0'], loss_function='rmse_log')

# Top k trials - recovery

In [ ]:
prediction_dict = {}
predictions_dict['m1'] = single_fitting_cycle(**copy.deepcopy(config['fitting'])) 

In [ ]:
predictions_dict['m1']['trials_processed'] = forecast_all_trials(predictions_dict, train_fit='m1', 
                                                                 model=config['fitting']['model'], 
                                                                 forecast_days=config['forecast']['forecast_days'])

In [ ]:
get_top_k_with_recovery_loss(predictions_dict)

In [ ]:
import pandas as pd

df = pd.DataFrame(best_params)
df

In [ ]:
df2 = pd.DataFrame(train_losses)
df2 = df2.add_prefix('trainloss_')
df2 = df2.set_index(df.index)

df3 = pd.DataFrame(val_losses)
df3 = df3.add_prefix('valloss_')
df3 = df3.set_index(df.index)

df2

In [ ]:
df_final = df.join(df2)
df_final = df_final.join(df3)

In [ ]:
df_final.to_csv('../../misc/reports/recovered_params3.csv')